In [ ]:
!git clone https://github.com/yu4u/age-gender-estimation.git

In [ ]:
%cd age-gender-estimation/

In [ ]:
!mkdir "facepics"
!mkdir "outputs"

In [ ]:
!cp "/content/drive/My Drive/yoloface/cfg" "cfg" -r
!cp "/content/drive/My Drive/yoloface/model-weights" "model-weights" -r
!cp "/content/drive/My Drive/yoloface/samples" "samples" -r
!cp "/content/drive/My Drive/yoloface/utils.py" "utils.py"

In [ ]:
# !python demo.py

In [ ]:
from pathlib import Path
import cv2
import dlib
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from google.colab.patches import cv2_imshow


def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)


@contextmanager
def video_capture(*args, **kwargs):
    cap = cv2.VideoCapture(*args, **kwargs)
    try:
        yield cap
    finally:
        cap.release()


def yield_images():
    # capture video
    with video_capture("/content/shakira.mp4") as cap:
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        while True:
            # get video frame
            ret, img = cap.read()

            if not ret:
                raise RuntimeError("Failed to capture image")

            yield img

depth = 16
k = 8
weight_file = "/content/age-gender-estimation/pretrained_models/weights.28-3.73.hdf5"
margin = 0.4
image_dir = None

if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                            file_hash=modhash, cache_dir=str(Path(__file__).resolve().parent))

# for face detection
detector = dlib.get_frontal_face_detector()

# load model and weights
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)

image_generator = yield_images_from_dir(image_dir) if image_dir else yield_images()

for img in image_generator:
    input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)

    # detect faces using dlib detector
    detected = detector(input_img, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))

    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))

        # predict ages and genders of the detected faces
        results = model.predict(faces)
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()

        # draw results
        for i, d in enumerate(detected):
            label = "{}, {}".format(int(predicted_ages[i]),
                                    "M" if predicted_genders[i][0] < 0.5 else "F")
            draw_label(img, (d.left(), d.top()), label)

    # cv2.imshow("result", img)
    cv2_imshow(img)
    key = cv2.waitKey(-1) if image_dir else cv2.waitKey(30)

    if key == 27:  # ESC
        break

INTERFACING WITH OLD CODE

In [ ]:
from utils import *
import cv2
import numpy as np


from pathlib import Path
import cv2
import dlib
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from google.colab.patches import cv2_imshow

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=1.0, thickness=3):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

Using TensorFlow backend.


In [ ]:
def iou(box1, box2):
    """
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """
    # Assign variable names to coordinates for clarity
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (yi1, xi1, yi2, xi2) coordinates of the intersection of box1 and box2. Calculate its Area.
    ### START CODE HERE ### (≈ 7 lines)
    xi1 = max(box1_x1,box2_x1)
    yi1 = max(box1_y1,box2_y1)
    xi2 = min(box1_x2,box2_x2)
    yi2 = min(box1_y2,box2_y2)
    inter_width = xi2-xi1
    inter_height = yi2-yi1
    inter_area = max(inter_width,0) * max(inter_height,0)
    ### END CODE HERE ###    

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    ### START CODE HERE ### (≈ 3 lines)
    box1_area = (box1_x2-box1_x1) * (box1_y2-box1_y1)
    box2_area = (box2_x2-box2_x1) * (box2_y2-box2_y1)
    union_area = (box1_area + box2_area) - inter_area
    ### END CODE HERE ###
    
    # compute the IoU
    ### START CODE HERE ### (≈ 1 line)
    iou = inter_area / union_area
    ### END CODE HERE ###
    return iou
class Sort(object):
  def __init__(self, max_age=1, iou_thresh =0.4):
    """
    Sets key parameters for SORT
    """
    #Age denotes the expiring of a track
    self.max_age = max_age
    self.iou_thresh = iou_thresh
    self.p_track = []
    self.frame_count = 0
    self.id = 0
    
  def update(self, dets=np.empty((0, 5))):
    # print(dets)
    self.frame_count += 1
    # self.p_track[:]['age'] += 1
    updated_trk = []
    # if self.id==0 and dets.size != 0:
    if not self.p_track and dets.size != 0:
      #Initialise first detections as Previous Trackers & if not first add with previous id(When p_track becomes empty)
      self.p_track = [{'bbox':list(det),'id':k + self.id,'age':0,'frame_count':1} for k,det in enumerate(dets,1)]
      self.id = self.p_track[-1]['id']
    elif self.p_track and dets.size != 0:
      # print("####")
      # print(self.p_track)
      new_p = self.p_track
      for det in dets:
        try:
          best_match_trk = max(new_p,key=lambda x:iou(det,x['bbox']))
        except ValueError:
          print("################################### VALUE ERROR ####################################################\n"+"det: ",end=" ")
          print(det)
          print("prev trk: ",end=" ")
          print(self.p_track)
          continue
        # print(iou(det,best_match_trk['bbox']))
        if iou(det,best_match_trk['bbox']) >= self.iou_thresh :
          best_match_trk['bbox'] = det
          best_match_trk['frame_count'] += 1
          # print(best_match_trk)
          #reset age
          best_match_trk['age'] = 0
          updated_trk.append(best_match_trk)
        else:
          #New non matched detection is assigned to new track
          self.id += 1
          new_trk = {'bbox':det,'id':self.id,'age':0,'frame_count':1}
          updated_trk.append(new_trk)
          if best_match_trk['age'] <= self.max_age:
            best_match_trk['age'] += 1
            updated_trk.append(best_match_trk)
        #removing the used best_match_trk from p_track
        # del dets[dets.index(best_match)]
        # print(self.p_track[self.p_track.index(best_match_trk)])
        # print("before:")
        # print(self.p_track)
        # print(best_match_trk)
        # print(self.p_track.index(best_match_trk))
        # del self.p_track[self.p_track.index(best_match_trk)]
        # self.p_track.remove(best_match_trk)
        self.p_track = list(filter(lambda x: x['id'] != best_match_trk['id'] and x['age'] < self.max_age, self.p_track)) 
        # print("after:")
        # print(self.p_track)        
      #increase age of previous tracks
      for trk in self.p_track:
        trk['age'] += 1
      self.p_track = self.p_track + updated_trk
    elif self.p_track and dets.size == 0:
      new_p = []
      for trk in self.p_track:
        trk['age'] += 1
        if(trk['age'] < self.max_age):
          new_p.append(trk)
      self.p_track = new_p
      return []
        
      
    # print("#######",end = " ")
    # print(self.p_track,end="#######")
    return self.p_track




In [ ]:
model_cfg = './cfg/yolov3-face.cfg'
model_weights = './model-weights/yolov3-wider_16000.weights'
# image = './samples/outside_000001.jpg'
video = './samples/sss.mp4'
# output_dir = '/content/drive/My Drive/yoloface/outputs/'
output_dir = '/content/age-gender-estimation/outputs/'

IOU_THRESH = 0.35
MAX_AGE = 3
#Trackers(Here faces) with given frame count value is chosen for age and gender processing
FRAME_COUNT_THRESH = 6
# DETECT_INTERVAL = 2
FACE_HEIGHT = 50
FACE_WIDTH = 50
# print the arguments
print('----- info -----')
print('[i] The config file: ',model_cfg)
print('[i] The weights of model file: ',model_weights)
# print('[i] Path to image file: ',image)
print('[i] Path to video file: ',video)
print('[i] IOU Threshold value: ',IOU_THRESH)
print('[i] Max age value: ',MAX_AGE)
print('[i] Frame count threshold value: ',FRAME_COUNT_THRESH)
# print('[i] Detect interval: ',DETECT_INTERVAL)
print('[i] Minimum face height required: ',FACE_HEIGHT)
print('[i] Minimum face width required: ',FACE_WIDTH)


#Read config net
net = cv2.dnn.readNetFromDarknet(model_cfg,model_weights)
print('###########################################################\n')


#VIDEO
cap = cv2.VideoCapture(video)
output_file = video[:-4].rsplit('/')[-1] + '_keras.avi'
output_file = output_dir + output_file

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

video_writer = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc(*'XVID'),cap.get(cv2.CAP_PROP_FPS), (
                                          round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                          round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))


# print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(cap.get(cv2.CAP_PROP_FPS))
#create instance of SORT
mot = Sort(MAX_AGE,IOU_THRESH)

----- info -----
[i] The config file:  ./cfg/yolov3-face.cfg
[i] The weights of model file:  ./model-weights/yolov3-wider_16000.weights
[i] Path to video file:  ./samples/sss.mp4
[i] IOU Threshold value:  0.35
[i] Max age value:  3
[i] Frame count threshold value:  6
[i] Minimum face height required:  50
[i] Minimum face width required:  50
###########################################################

23.976023976023978


In [ ]:
depth = 16
k = 8
weight_file = "/content/age-gender-estimation/pretrained_models/weights.28-3.73.hdf5"
margin = 0.4
image_dir = None
# load model and weights
img_size = 64
# img_size = 32
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)


In [ ]:
#Frame counter for detect interval
# c=0
while True:
  
  has_frame, frame = cap.read()
  if not has_frame:
      print('[i] ==> Done processing!!!')
      #print('[i] ==> Output file is stored at', os.path.join(args.output_dir, output_file))
      break
  # frame = cv2.resize(frame, (0, 0), fx=0.7, fy=0.7)
  # Resize, Convert BGR to HSV
  # if ((IMG_HEIGHT, IMG_WIDTH) != frame.shape[0:2]):
  #     frame = cv2.resize(frame, dsize=(IMG_WIDTH, IMG_HEIGHT), fx=0, fy=0)
  # else:
  #     frame = frame
  # if(c % DETECT_INTERVAL==0):
  new_frame = frame.copy()
  img_h, img_w, _ = np.shape(frame)

  blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT),[0, 0, 0], 1, crop=False)
  net.setInput(blob)
  # Runs the forward pass to get output of the output layers
  outs = net.forward(get_outputs_names(net))
  # Remove the bounding boxes with low confidence
  faces = post_process(frame, outs,CONF_THRESHOLD, NMS_THRESHOLD)
  faces = np.array(faces) 
  
  # faces_resized = []

  if(faces.shape[0]>0):
    final_faces = faces[:,:4]
    
  else:
    final_faces = faces
  trackers = mot.update(final_faces)
  # print("Final faces:",end=" ")
  # print(final_faces)
  # print("$$$$$$$$$$$$$ ") 
  # print(trackers)
  # print("&&&&&&&&&&&&&&&&&&")
  print("#",end="")
  faces_resized = np.empty((len(trackers), img_size, img_size, 3))

  for i,trk in enumerate(trackers):
    box = np.array(trk['bbox'])
    id = int(trk['id'])
    frame_count = trk['frame_count']
    d = box.astype(np.int32)
    (startX, startY, endX, endY) = (d[0],d[1],d[2],d[3])
    face_h = endY - startY
    face_w = endX - startX

    x1, y1, x2, y2, w, h = d[0], d[1], d[2] + 1, d[3] + 1, face_w, face_h
    xw1 = max(int(x1 - margin * w), 0)
    yw1 = max(int(y1 - margin * h), 0)
    xw2 = min(int(x2 + margin * w), img_w - 1)
    yw2 = min(int(y2 + margin * h), img_h - 1)
    # cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.rectangle(frame, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
    # faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))




    # if(frame_count==FRAME_COUNT_THRESH and (face_h > FACE_HEIGHT and face_w > FACE_WIDTH)):
    # crop_face = new_frame[startY:endY, startX:endX]
    crop_face_shape = new_frame[yw1:yw2 + 1, xw1:xw2 + 1, :]

    crop_face = cv2.resize(new_frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
    # crop_face = cv2.resize( new_frame[startY:endY, startX:endX], (img_size,img_size))

    print(crop_face_shape.shape)



    faces_resized[i,:,:,:] = crop_face

    if(frame_count == FRAME_COUNT_THRESH):  
      try:
        cv2.imwrite("{0}/{1}.jpg".format("facepics",id),crop_face)
        # cv2_imshow(crop_face)
      except:
        print("#########################Image error####################")
        continue


  # predict ages and genders of the detected faces
  # print(faces_resized)


  if(faces_resized.size > 0):
    results = model.predict(faces_resized)
    predicted_genders = results[0]
    ages = np.arange(0, 101).reshape(101, 1)
    predicted_ages = results[1].dot(ages).flatten()
    for i,trk in enumerate(trackers):
      # if(frame_count==FRAME_COUNT_THRESH and (face_h > FACE_HEIGHT and face_w > FACE_WIDTH)):
      d = np.array(trk['bbox']).astype(np.int32)
      id = int(trk['id'])
      label = "{}, {}, {}".format(id,int(predicted_ages[i]),"M" if predicted_genders[i][0] < 0.5 else "F")
      draw_label(frame, (d[0], d[1]), label)



    # print(results)
    # print("#" * 20)
    # print(predicted_genders)
    # print("#" * 20)
    # print(ages)
    # print("#" * 20)
    # print(predicted_ages)
    # print("#" * 20)
    # draw results



  # cv2_imshow(frame)
  video_writer.write(frame.astype(np.uint8))
  # c+=1



In [ ]:
!zip -r /content/file.zip /content/age-gender-estimation/facepics

In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(results)

In [ ]:
cap.release()
video_writer.release()

In [ ]:
new_out = "/content/drive/My Drive/yoloface/facepics/" + video[:-4].rsplit('/')[-1] + '_pics'

In [ ]:
print(new_out)
print(output_file)

/content/drive/My Drive/yoloface/facepics/pro_sample_pics
/content/age-gender-estimation/outputs/pro_sample_keras.avi


In [ ]:
!cp "facepics" "/content/drive/My Drive/yoloface/facepics/sushant_pics" -r

In [ ]:
!cp "/content/age-gender-estimation/outputs/sushant_keras.avi" "/content/drive/My Drive/yoloface/outputs/sushant_keras.avi" -r

In [ ]:
rm facepics -r

In [ ]:
mkdir "facepics"